# Pytorch Convolution Neural Network 

In this notebook I am going to build a convolution neural network using torch.nn module. We are going to train the CNN on MNIST data set . 

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

test.csv
train.csv



Here first we have to import several torch libraries like torch.nn , torch.data.utils, torchvision etc.

In [3]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

A very important part in building any deep NN in pytorch is first building a class conating the data . This data.datatset class wil contain  init() , getitem() , len() . After defining the class we create an object of that class . Then call the dataloader on the data with    batch_size = 50 .  

In [4]:
class Mnist(data.Dataset):
    def __init__(self):
        train_X = pd.read_csv("../input/train.csv")
        train_Y = train_X.label.as_matrix().tolist()
        train_X = train_X.drop("label",axis=1).as_matrix().reshape(42000,1,28,28)
        self.datalist = train_X
        self.labellist = train_Y


    def __getitem__(self, index):
        return torch.Tensor(self.datalist[index].astype(float)), self.labellist[index]

    def __len__(self):
        return self.datalist.shape[0]

train_data = Mnist()
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=50, 
                                           shuffle=True,
                                           num_workers=2)

Now we create the nn.module class CNN . It must contain two methods init ( ) and forward ( ) . In init ( ) we define the sequential steps of the CNN model using Conv2d , ReLU , MaxPool2d etc. In forward method we do the forward propagation part . 

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 20, kernel_size=5),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(12*12*20, 10)


    def forward(self, x):
        out = self.layer1(x)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out
    
cnn = CNN()

After craeting an instance cnn of class CNN now we define the loss function and the optimizer technique , which is SGD here with learning rate = .004

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cnn.parameters(), lr=0.0004)

Here we start the actual CNN implemetation through iterating over allthe examples . Here we are only running 3 iterations and in each epoch we first call cnn on each image then we do the backward propagation and update the parameters . 

In [7]:
for epoch in range(3):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)

        optimizer.zero_grad()
        outputs = cnn(images)

        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' %(epoch+1, 5, i+1, len(train_data)//50,  loss.data[0]))


Epoch [1/5], Iter [100/840] Loss: 2.0436
Epoch [1/5], Iter [200/840] Loss: 1.7354
Epoch [1/5], Iter [300/840] Loss: 1.5608
Epoch [1/5], Iter [400/840] Loss: 1.2858
Epoch [1/5], Iter [500/840] Loss: 1.0737
Epoch [1/5], Iter [600/840] Loss: 1.1584
Epoch [1/5], Iter [700/840] Loss: 0.9174
Epoch [1/5], Iter [800/840] Loss: 0.8727
Epoch [2/5], Iter [100/840] Loss: 0.8155
Epoch [2/5], Iter [200/840] Loss: 0.8093
Epoch [2/5], Iter [300/840] Loss: 0.8180
Epoch [2/5], Iter [400/840] Loss: 0.5827
Epoch [2/5], Iter [500/840] Loss: 0.5634
Epoch [2/5], Iter [600/840] Loss: 0.6002
Epoch [2/5], Iter [700/840] Loss: 0.5440
Epoch [2/5], Iter [800/840] Loss: 0.5681
Epoch [3/5], Iter [100/840] Loss: 0.5709
Epoch [3/5], Iter [200/840] Loss: 0.6514
Epoch [3/5], Iter [300/840] Loss: 0.4700
Epoch [3/5], Iter [400/840] Loss: 0.4380
Epoch [3/5], Iter [500/840] Loss: 0.5014
Epoch [3/5], Iter [600/840] Loss: 0.5057
Epoch [3/5], Iter [700/840] Loss: 0.5604
Epoch [3/5], Iter [800/840] Loss: 0.4555


So ,finally we see that in just 3 epochs the loss value has come down to 0.45 from 2.04 . 